In [45]:
import pybaseball
from pybaseball import statcast
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timezone
from bs4 import BeautifulSoup
import io
import requests
import unicodedata

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")
pybaseball.cache.enable()

from RosterScraper import RosterScraper

In [46]:
url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pub?output=csv'
res = requests.get(url)
ID = pd.read_csv(io.BytesIO(res.content), sep=',')
ID.dropna(subset=['MLBID'], inplace=True)
ID['MLBID'] = ID['MLBID'].astype(int)

Rosters = RosterScraper()
BID = Rosters[Rosters["Position"] == "Batter"]
PID = Rosters[Rosters["Position"] == "Pitcher"]

In [47]:
def convert_name(name):
    if name == 'Rockies':
        return 'COL'
    elif name == 'Reds':
        return 'CIN'
    elif name == 'Mariners':
        return 'SEA'
    elif name == 'Nationals':
        return 'WSH'
    elif name == 'Yankees':
        return 'NYY'
    elif name == 'Astros':
        return 'HOU'
    elif name == 'Red Sox':
        return 'BOS'
    elif name == 'Athletics':
        return 'OAK'
    elif name == 'Mets':
        return 'NYM'
    elif name == 'Braves':
        return 'ATL'
    elif name == 'Giants':
        return 'SF'
    elif name == 'Brewers':
        return 'MIL'
    elif name == 'Rays':
        return 'TB'
    elif name == 'Royals':
        return 'KC'
    elif name == 'White Sox':
        return 'CWS'
    elif name == 'Cubs':
        return 'CHC'
    elif name == 'Angels':
        return 'LAA'
    elif name == 'Tigers':
        return 'DET'
    elif name == 'Diamondbacks':
        return 'ARI'
    elif name == 'Guardians':
        return 'CLE'
    elif name == 'Orioles':
        return 'BAL'
    elif name == 'Twins':
        return 'MIN'
    elif name == 'Marlins':
        return 'MIA'
    elif name == 'Phillies':
        return 'PHI'
    elif name == 'Rangers':
        return 'TEX'
    elif name == 'Dodgers':
        return 'LAD'
    elif name == 'Padres':
        return 'SD'
    elif name == 'Pirates':
        return 'PIT'
    elif name == 'Blue Jays':
        return 'TOR'
    elif name == 'Cardinals':
        return 'STL'
    else:
        return np.nan
    
def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

def replace_special_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

In [48]:
def getDKData2024():
    
    eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
    todaysdate = eastern_time.strftime("%m-%d-%Y")
    url = 'https://rotogrinders.com/lineups/mlb?site=draftkings'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    gamelist = []
    gamecards = soup.findAll("div", {"class": "game-card-teams"})
    for x in gamecards:
        twoteams = x.findAll("span", {"class": "team-nameplate-mascot"})
        roadteam = convert_name(twoteams[0].text)
        hometeam = convert_name(twoteams[1].text)
        gamekey = "{}@{}".format(roadteam,hometeam)
        gamelist.append(gamekey)

    matchupsdf = pd.DataFrame()
    for game in gamelist:
        roadteam = game.split("@")[0]
        hometeam = game.split("@")[1]
        thisdf1 = pd.DataFrame({"Team": roadteam, "Opp": hometeam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        thisdf2 = pd.DataFrame({"Team": hometeam, "Opp": roadteam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        matchupsdf = pd.concat([matchupsdf,thisdf1,thisdf2])
        
    oppdict = dict(zip(matchupsdf.Team,matchupsdf.Opp))
    hometeamdict = dict(zip(matchupsdf.Team,matchupsdf.HomeTeam))
    roadteamdict = dict(zip(matchupsdf.Team,matchupsdf.RoadTeam))

    disabled_span_list = []
    for span in soup.findAll("span", {"class": "player-nameplate disabled"}):
        for a in span.findAll("a"):
            disabled_span_list.append(a.text)

    spdata = pd.DataFrame()
    for div in soup.findAll("span", {"class": "player-nameplate", "data-position": "SP"}):
        if "TBD" in str(div):
            playername = "TBD"
            pos = "SP"
            sal = 0
        else:
            for a in div.findAll('a', {'class': 'player-nameplate-name'}):
                playername = a.text.strip()

            strdiv = str(div)
            pos = strdiv[strdiv.find("data-position")+15:strdiv.find("data-salary")-2]
            sal = strdiv[strdiv.find("data-salary")+13:strdiv.find("<div class = 'player-nameplate-info'>")-3]
        try:
            ownership = strdiv[strdiv.find('<span class="small muted" data-auth="502">') + 42:strdiv.find('%')]
            ownership = ownership.replace("</span>", "")
            ownership = ownership.replace("</span", "")
            ownership = ownership.replace("</div>", "")
            ownership = ownership.replace(" ", "")
        except:
            ownership = np.nan

        thisspdata = pd.DataFrame([[playername, sal, ownership]], columns = ["Player", "Salary", "Ownership"])
        spdata = pd.concat([spdata, thisspdata])

    spdata2 = pd.merge(spdata, PID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "PitcherTeam"})
    spdata3 = pd.merge(spdata2, matchupsdf[["Team", "Opp"]], left_on = ["PitcherTeam"], right_on = ["Team"], how = "left").drop(columns = ["Team"])

    opp_spname_dict = dict(zip(spdata3.Opp, spdata3.Player))
    opp_spsal_dict = dict(zip(spdata3.Opp, spdata.Salary))
    opp_spown_dict = dict(zip(spdata3.Opp, spdata3.Ownership))

    ludf = pd.DataFrame()
    
    for li in soup.findAll("li", {"class": "lineup-card-player"}):
        for a in li.findAll("a", {"class": ["player-nameplate-name", "player-nameplate disabled"]}):
            playername = a.text

        listring = str(li)
        for span in li.find("span", {"class": "small"}):
            luspot = span.text
            luspot = luspot.replace("\n", "")
            luspot = luspot.strip()
            luspot = int(luspot)
        pos = listring[listring.find("data-position")+15:listring.find("data-salary")-2]
        sal = listring[listring.find("data-salary")+13:listring.find("<span class='small'>")-3]
        ownership = ownership.replace("</span>", "")
        ownership = ownership.replace("</span", "")
        ownership = ownership.replace("</li", "")
        ownership = ownership.replace("</div>", "")
        ownership = ownership.replace(" ", "")

        try:
            sal = int(sal)
        except:
            sal = 0
        thisludf = pd.DataFrame([[playername, luspot, sal, ownership]], columns = ["Player", "Spot", "Sal", "Ownership"])
        ludf = pd.concat([ludf, thisludf])

    ludf2 = pd.merge(ludf, BID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "BatterTeam"})
    ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')

    ludf2_teamlist = list(ludf2["BatterTeam"])

    dhteams = []
    for x in ludf2_teamlist:
        if ludf2_teamlist.count(x) > 11:
            if x in dhteams:
                pass
            else:
                dhteams.append(x)

    extract_dh = ludf2[ludf2["BatterTeam"].isin(dhteams)]
    new_ludf2 = ludf2[~ludf2["BatterTeam"].isin(dhteams)]

    new_team_list = []
    runcounter = 0

    for x in list(extract_dh["BatterTeam"].astype(str)):
        if runcounter < 18:
            new_team_list.append(x)
            runcounter += 1
        else:
            new_team_list.append(x+"2")
            runcounter += 1

    extract_dh["BatterTeam"] = new_team_list

    ludf2 = pd.concat([extract_dh, new_ludf2])
    ludf2["Opp"] = ludf2["BatterTeam"].map(oppdict)
    ludf2["HomeTeam"] = ludf2["BatterTeam"].map(hometeamdict)
    ludf2["RoadTeam"] = ludf2["BatterTeam"].map(roadteamdict)
    ludf2['SP'] = ludf2['BatterTeam'].map(opp_spname_dict)
    ludf2['SPSal'] = ludf2['BatterTeam'].map(opp_spsal_dict)
    ludf2['SPOwnership'] = ludf2['BatterTeam'].map(opp_spown_dict)
    ludf2['Date'] = todaysdate
    ludf2['Time'] = np.nan

    ludf3 = ludf2[['BatterTeam','RoadTeam','HomeTeam','Time','Spot','Player','Sal','Ownership','Date', "SP"]]

    dkdata = ludf3.copy()

    try:
        checknan = dkdata[["BatterTeam", "SP"]]
        getnans = checknan[["SP"].isna()]
        if len(getnans) == 0:
            nonans = 1
            nanmapdict = {}
        else:
            nonans = 0
            getnans["SP"] = disabled_span_list
            nanmapdict = dict(zip(getnans.Team, getnans.SP))
    except:
        pass

    try:
        dkdata["SP"] = np.where(dkdata["SP"].isna(), dkdata["BatterTeam"].map(nanmapdict), dkdata["SP"])
    except:
        pass
    
    for i in range(1, len(dkdata) - 1):
        if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i-1, 'BatterTeam']:
            if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i+1, 'BatterTeam']:
                dkdata.loc[i, 'BatterTeam'] = np.nan
                dkdata.loc[i, 'HomeTeam'] = np.nan
                dkdata.loc[i, 'RoadTeam'] = np.nan
                dkdata.loc[i, 'SP'] = np.nan

    
    dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')
    dkdata = dkdata.drop_duplicates(subset = ["BatterTeam", "SP"], keep = "first")
    dkdata = dkdata.drop(columns = ["Time", "Sal", "Ownership"])

    dkdata['BatterTeam'] = dkdata['BatterTeam'].replace('ARI', 'AZ')
    dkdata['RoadTeam'] = dkdata['RoadTeam'].replace('ARI', 'AZ')
    dkdata['HomeTeam'] = dkdata['HomeTeam'].replace('ARI', 'AZ')

    dkdata['Date'] = pd.to_datetime(dkdata['Date'])
    dkdata['Date'] = dkdata['Date'].dt.strftime('%Y-%m-%d')
    dkdata = dkdata.set_index("Date")
    dkdata = dkdata[["BatterTeam", "RoadTeam", "HomeTeam", "SP"]]

    return(dkdata)

In [49]:
#statcast(start_dt = "2022-04-07", end_dt = "2022-10-05")
#statcast(start_dt = "2023-03-30", end_dt = "2023-10-01")
savant2022 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2022.csv")
savant2023 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2023.csv")

In [50]:
#pd.set_option('display.max_columns', None)
combined1 = pd.concat([savant2022, savant2023])
combined1['game_date'] = pd.to_datetime(combined1['game_date'])
combined1['game_date'] = combined1['game_date'].dt.strftime('%Y-%m-%d')
combined1['BatterTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['away_team'], combined1['home_team'])
combined1['PitcherTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['home_team'], combined1['away_team'])
combined1["player_name"] = combined1["player_name"].apply(flip_names)
combined1["player_name"] = combined1["player_name"].apply(replace_special_chars)

In [53]:
# Group by game and team identifiers
groupby_cols = ['game_date', 'BatterTeam', 'away_team', 'home_team']

# Function to keep only the starter's data
def keep_starter(group):
    starter_name = group['player_name'].iloc[0]
    return group[group['player_name'] == starter_name]

def count_outs(x):
    single_outs = ['other_out', 'strikeout', 'field_out', 'fielders_choice', 'fielders_choice_out']
    double_outs = ['double_play', 'strikeout_double_play', 'grounded_into_double_play']
    triple_outs = ['triple_play']
    
    outs = (x.isin(single_outs)).sum() + 2 * (x.isin(double_outs)).sum() + 3 * (x.isin(triple_outs)).sum()
    return outs

# Apply the function to each group
combined2 = combined1[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "BatterTeam", "MLBNAME", "events", "description", "bb_type", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_run_exp"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
df_starters_only = combined2.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_71964/259392456.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_starters_only = combined2.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)


In [92]:
Train1 = df_starters_only.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean")).reset_index().fillna(0)

In [132]:
Train2 = Train1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

Train2["P/AB"] = round((Train2['Pitches'] / Train2['AB']), 2)
Train2['K%'] = round((Train2['K'] / Train2['AB']) * 100, 2)
Train2['BB%'] = round((Train2['BB'] / Train2['AB']) * 100, 2)
Train2['K-BB%'] = Train2["K%"] - Train2["BB%"]
Train2['Ball%'] = round((Train2['Balls'] / Train2['Pitches']) * 100, 2)
Train2['Strike%'] = round((Train2['Strikes'] / Train2['Pitches']) * 100, 2)
Train2['CS%'] = round((Train2['CS'] / Train2['Pitches']) * 100, 2)
Train2['Whiff%'] = round((Train2['Whiff'] / Train2['Pitches']) * 100, 2)
Train2["CSW"] = Train2["CS"] + Train2["Whiff"]
Train2['CSW%'] = round((Train2['CSW'] / Train2['Pitches']) * 100, 2)
Train2 = Train2.drop(columns = ["CSW"])

In [133]:
Train3 = Train2.rename(columns={'away_team': 'RoadTeam', 'home_team': 'HomeTeam', "player_name": "SP"})
Train3 = Train3.drop(columns = ["game_date", "Balls", "HBP", "BB", "CS", "Whiff", "Strikes"])
Train3

,BatterTeam,RoadTeam,HomeTeam,SP,p_throws,Pitches,AB,PA,Outs,K,...,RunExp,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%
0,ATL,CIN,ATL,Tyler Mahle,R,84,20,18,15,7,...,-0.028863,4.20,35.00,10.00,25.00,39.29,46.43,19.05,15.48,34.52
1,AZ,SD,AZ,Yu Darvish,R,92,22,17,17,3,...,-0.040531,4.18,13.64,18.18,-4.54,44.57,39.13,17.39,10.87,28.26
2,CHC,MIL,CHC,Corbin Burnes,R,83,22,16,12,4,...,-0.020852,3.77,18.18,13.64,4.54,42.17,40.96,18.07,13.25,31.33
3,CIN,CIN,ATL,Max Fried,L,84,26,23,16,5,...,0.016518,3.23,19.23,3.85,15.38,33.33,44.05,16.67,11.90,28.57
4,CLE,CLE,KC,Zack Greinke,R,84,22,20,16,1,...,-0.038159,3.82,4.55,4.55,0.00,33.33,42.86,22.62,5.95,28.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9509,STL,CIN,STL,Hunter Greene,R,45,9,8,5,3,...,0.016376,5.00,33.33,0.00,33.33,42.22,46.67,13.33,11.11,24.44
9510,TB,TB,TOR,Wes Parsons,R,60,15,12,7,2,...,-0.021352,4.00,13.33,13.33,0.00,41.67,41.67,20.00,8.33,28.33
9511,TEX,TEX,SEA,George Kirby,R,68,18,18,17,6,...,-0.062775,3.78,33.33,0.00,33.33,30.88,51.47,16.18,14.71,30.88
9512,TOR,TB,TOR,Jacob Lopez,L,65,21,19,14,3,...,0.109993,3.10,14.29,4.76,9.53,35.38,40.00,12.31,7.69,20.00


In [113]:
BatterTrain1 = combined2.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean")).reset_index().fillna(0)

In [124]:
BatterTrain2 = BatterTrain1.groupby(["BatterTeam", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

BatterTrain2["P/AB"] = round((BatterTrain2['Pitches'] / BatterTrain2['AB']), 2)
BatterTrain2['K%'] = round((BatterTrain2['K'] / BatterTrain2['AB']) * 100, 2)
BatterTrain2['BB%'] = round((BatterTrain2['BB'] / BatterTrain2['AB']) * 100, 2)
BatterTrain2['K-BB%'] = BatterTrain2["K%"] - BatterTrain2["BB%"]
BatterTrain2['Ball%'] = round((BatterTrain2['Balls'] / BatterTrain2['Pitches']) * 100, 2)
BatterTrain2['Strike%'] = round((BatterTrain2['Strikes'] / BatterTrain2['Pitches']) * 100, 2)
BatterTrain2['CS%'] = round((BatterTrain2['CS'] / BatterTrain2['Pitches']) * 100, 2)
BatterTrain2['Whiff%'] = round((BatterTrain2['Whiff'] / BatterTrain2['Pitches']) * 100, 2)
BatterTrain2["CSW"] = BatterTrain2["CS"] + BatterTrain2["Whiff"]
BatterTrain2['CSW%'] = round((BatterTrain2['CSW'] / BatterTrain2['Pitches']) * 100, 2)
BatterTrain2 = BatterTrain2.drop(columns = ["CSW"])

In [130]:
BatterTrain2.sort_values("K%").head(15)
BatterTrain3 = BatterTrain2.rename(columns={'away_team': 'RoadTeam', 'home_team': 'HomeTeam', "player_name": "SP"})
BatterTrain3 = BatterTrain3.drop(columns = ["Balls", "HBP", "BB", "CS", "Whiff", "Strikes", "K"])
BatterTrain3.sort_values("wOBA", ascending = False).head(15)

,BatterTeam,p_throws,Pitches,AB,PA,Outs,xBA,xwOBA,wOBA,RunExp,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%
0,ATL,L,11450,2884,2518,1869,0.238545,0.321647,0.363159,0.019519,3.97,22.50,8.95,13.55,36.38,46.48,14.33,13.08,27.41
20,HOU,L,12857,3420,2966,2193,0.235586,0.308359,0.357018,0.019110,3.76,17.08,8.92,8.16,36.36,44.13,14.98,10.34,25.32
27,LAD,R,32988,8339,7181,5350,0.225916,0.303437,0.350312,0.016525,3.96,20.82,9.47,11.35,37.38,45.33,16.26,11.33,27.58
36,NYY,L,9532,2405,2063,1579,0.226086,0.315712,0.350062,0.015880,3.96,21.87,9.69,12.18,36.83,46.23,17.47,11.78,29.25
1,ATL,R,34676,8814,7776,5861,0.231169,0.307888,0.348241,0.016344,3.93,22.77,7.84,14.93,35.93,46.78,14.34,13.54,27.88
50,STL,L,10035,2566,2215,1668,0.224712,0.297918,0.347233,0.018102,3.91,21.28,9.35,11.93,35.80,46.76,16.53,11.95,28.48
54,TEX,L,12185,3197,2782,2062,0.219712,0.290202,0.346903,0.011188,3.81,21.71,8.91,12.80,36.46,45.51,15.31,12.46,27.76
40,PHI,L,13676,3478,3023,2273,0.223278,0.290812,0.344192,0.012087,3.93,23.78,8.34,15.44,35.70,47.13,16.35,12.96,29.31
26,LAD,L,13046,3241,2802,2125,0.220568,0.296416,0.343382,0.017416,4.03,22.03,8.98,13.05,37.08,46.16,15.87,11.80,27.66
6,BOS,L,11221,2816,2435,1810,0.217566,0.281931,0.342489,0.022330,3.98,23.19,8.81,14.38,35.66,47.52,17.23,12.53,29.76


In [57]:
TodaysData = getDKData2024()
TodaysData

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_71964/3305814355.py:95: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_71964/3305814355.py:164: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')


,BatterTeam,RoadTeam,HomeTeam,SP
Date,,,,
2024-07-12,LAD,LAD,DET,Tarik Skubal
2024-07-12,DET,LAD,DET,James Paxton
2024-07-12,OAK,OAK,PHI,Ranger Suarez
2024-07-12,PHI,OAK,PHI,Hogan Harris
2024-07-12,CLE,CLE,TB,Taj Bradley
2024-07-12,TB,CLE,TB,Carlos Carrasco
2024-07-12,NYY,NYY,BAL,Cade Povich
2024-07-12,BAL,NYY,BAL,Gerrit Cole
2024-07-12,MIA,MIA,CIN,Carson Spiers


In [58]:
eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
savant2024 = statcast(start_dt = "2024-03-28", end_dt = eastern_time.strftime("%Y-%m-%d"))
savant2024['game_date'] = pd.to_datetime(savant2024['game_date'])
savant2024['game_date'] = savant2024['game_date'].dt.strftime('%Y-%m-%d')
savant2024['BatterTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['away_team'], savant2024['home_team'])
savant2024['PitcherTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['home_team'], savant2024['away_team'])
savant2024 = pd.merge(savant2024, ID[["MLBID", "MLBNAME"]], left_on = 'batter', right_on = 'MLBID', how = 'left')
savant2024.dropna(subset=['MLBNAME'], inplace=True)
savant2024 = savant2024.drop_duplicates(subset = ["pitch_type", "game_date", "release_speed", "release_pos_x", "release_pos_z", "player_name"], keep='first')
savant2024["player_name"] = savant2024["player_name"].apply(flip_names)
savant2024 = savant2024[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "pitch_type", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "events", "description", "bb_type", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
savant2024 = savant2024.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)
savant2024["player_name"] = savant2024["player_name"].apply(replace_special_chars)

This is a large query, it may take a moment to complete


100%|██████████| 107/107 [00:58<00:00,  1.82it/s]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_71964/2056470012.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  savant2024 = savant2024.groupby(groupby

In [59]:
Season1 = savant2024.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean")).reset_index().fillna(0)

In [61]:
Season2 = Season1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

Season2

,game_date,BatterTeam,away_team,home_team,player_name,p_throws,Pitches,AB,PA,Outs,...,Balls,BB,CS,Whiff,Strikes,K,xBA,xwOBA,wOBA,RunExp
0,2024-03-28,AZ,COL,AZ,Kyle Freeland,L,49,17,16,7,...,15,1,8,6,20,2,0.442235,0.538703,0.676471,0.272008
1,2024-03-28,BAL,LAA,BAL,Patrick Sandoval,L,60,14,10,3,...,24,2,10,6,26,2,0.274357,0.359207,0.575,0.035
2,2024-03-28,BOS,BOS,SEA,Luis Castillo,R,91,24,19,13,...,33,2,17,6,42,5,0.24875,0.337412,0.395833,0.036904
3,2024-03-28,CHC,CHC,TEX,Nathan Eovaldi,R,88,22,19,17,...,33,1,11,6,38,3,0.236909,0.340148,0.290909,0.013162
4,2024-03-28,CIN,WSH,CIN,Josiah Gray,R,80,22,20,12,...,28,2,14,15,37,6,0.279273,0.401086,0.538636,0.024324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,2024-07-11,SEA,SEA,LAA,Jack Kochanowicz,R,60,16,13,7,...,19,0,10,5,28,1,0.352813,0.505063,0.59375,0.085339
2766,2024-07-11,SF,TOR,SF,Kevin Gausman,R,90,24,24,18,...,32,0,13,5,36,2,0.302833,0.33875,0.270833,-0.016753
2767,2024-07-11,TB,NYY,TB,Nestor Cortes,L,97,23,19,12,...,37,2,17,14,43,4,0.250957,0.417045,0.473913,0.067565
2768,2024-07-11,TOR,TOR,SF,Jordan Hicks,R,80,20,17,10,...,29,1,15,5,34,1,0.3155,0.439289,0.495,0.026606


In [62]:
Season3 = Season2.groupby(["player_name", "p_throws"]).agg(
    Games = ("Pitches", "size"),
    Pitches = ("Pitches", "sum"),
    AB = ("AB", "sum"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

Season3["P/AB"] = round(Season3["Pitches"] / Season3["AB"], 2)
Season3['K%'] = round((Season3['K'] / Season3['AB']) * 100, 2)
Season3['BB%'] = round((Season3['BB'] / Season3['AB']) * 100, 2)
Season3['K-BB%'] = Season3["K%"] - Season3["BB%"]
Season3['Ball%'] = round((Season3['Balls'] / Season3['Pitches']) * 100, 2)
Season3['Strike%'] = round((Season3['Strikes'] / Season3['Pitches']) * 100, 2)
Season3['CS%'] = round((Season3['CS'] / Season3['Pitches']) * 100, 2)
Season3['Whiff%'] = round((Season3['Whiff'] / Season3['Pitches']) * 100, 2)
Season3["CSW"] = Season3["CS"] + Season3["Whiff"]
Season3['CSW%'] = round((Season3['CSW'] / Season3['Pitches']) * 100, 2)
Season3 = Season3.drop(columns = ["CSW"])

In [63]:
# List of columns to keep unchanged
columns_to_keep = ["P/AB", "xBA", "wOBA", "xwOBA", "RunExp", "K%", "BB%", "K-BB%", "Ball%", "Strike%", "CS%", "Whiff%", "CSW%"]

# List of columns to divide
columns_to_divide = ["Pitches", "AB", "PA", "Outs", "HBP", "Balls", "BB", "CS", "Whiff", "Strikes", "K"]
divide_by_column = 'Games'

# Create the new DataFrame
Season4 = Season3.copy()

for col in columns_to_keep:
    Season4[col] = Season3[col]

for col in columns_to_divide:
    Season4[col] = Season3[col] / Season3[divide_by_column]

In [64]:
TodaysData.dropna(subset=['SP'], inplace=True)
TodaysData1 = pd.merge(TodaysData, Season4[['player_name', "p_throws", 'Pitches', 'AB', 'PA', 'Outs', 'HBP', 'Balls', 'BB', 'CS', 'Whiff', 'Strikes', 'K', "xBA", 'xwOBA', 'wOBA', 'RunExp', 'P/AB', 'K%', 'BB%', 'K-BB%', 'Ball%', 'Strike%', 'CS%', 'Whiff%', 'CSW%']], left_on = ['SP'], right_on = ['player_name'], how = 'left').drop(columns = ["player_name"])

# If no 2024 savant data exists then gives them the league averages from 2022-23
TrainMeans = Train3.drop(['BatterTeam', 'RoadTeam', "HomeTeam", "SP", "p_throws"], axis=1).mean()
TodaysData1 = TodaysData1.fillna(TrainMeans)

In [65]:
TodaysData1

,BatterTeam,RoadTeam,HomeTeam,SP,p_throws,Pitches,AB,PA,Outs,HBP,...,RunExp,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%
0,LAD,LAD,DET,Tarik Skubal,L,88.882353,23.764706,21.647059,17.117647,0.352941,...,-0.000663,3.74,30.69,4.7,25.99,30.58,52.42,17.34,17.14,34.48
1,DET,LAD,DET,James Paxton,L,84.3125,21.375,18.1875,14.125,0.0625,...,-0.00411,3.94,16.08,11.4,4.68,38.84,42.7,13.71,9.86,23.57
2,OAK,OAK,PHI,Ranger Suarez,L,87.444444,23.388889,20.888889,16.388889,0.277778,...,0.006126,3.74,24.23,4.99,19.24,34.37,46.82,17.41,10.67,28.08
3,PHI,OAK,PHI,Hogan Harris,L,90.714286,22.0,19.285714,14.571429,0.142857,...,0.023057,4.12,17.53,9.09,8.44,37.64,44.72,17.48,10.08,27.56
4,CLE,CLE,TB,Taj Bradley,R,90.363636,22.545455,19.818182,15.818182,0.090909,...,-0.004974,4.01,29.84,8.87,20.97,36.32,48.29,12.37,14.29,26.66
5,TB,CLE,TB,Carlos Carrasco,R,78.333333,21.6,19.133333,14.2,0.333333,...,0.038357,3.63,19.14,6.17,12.97,34.55,45.19,15.74,10.98,26.72
6,NYY,NYY,BAL,Cade Povich,L,84.5,20.833333,17.333333,12.833333,0.333333,...,0.042829,4.06,14.4,10.4,4.0,36.69,45.17,17.16,7.89,25.05
7,BAL,NYY,BAL,Gerrit Cole,R,78.5,19.75,17.25,12.75,0.5,...,0.049785,3.97,24.05,10.13,13.92,35.67,48.09,15.92,11.78,27.71
8,MIA,MIA,CIN,Carson Spiers,R,94.0,24.75,22.5,16.0,0.0,...,0.010257,3.8,17.17,5.05,12.12,34.84,44.41,16.22,9.04,25.27
9,CIN,MIA,CIN,Yonny Chirinos,R,79.25,20.5,18.5,12.25,0.25,...,0.029932,3.87,23.17,6.1,17.07,31.86,50.16,19.24,11.67,30.91


In [66]:
# Ensure Train5 and TodaysData2 are copies of Train4 and TodaysData1 respectively
Train5 = Train3.copy()
TodaysData2 = TodaysData1.copy()

# Dictionary to store the label encoders
label_encoders = {}

# Encode non-numeric columns in Train4
non_numeric_columns_train = Train5.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_train:
    label_encoder = LabelEncoder()
    Train5[col] = label_encoder.fit_transform(Train5[col])
    label_encoders[col] = label_encoder

# Ensure all non-numeric columns in Train4 are in TodaysData1
for col in non_numeric_columns_train:
    if col not in TodaysData2.columns:
        print(f"Warning: Column {col} from training data is not present in today's data.")
        # Adding the missing column with a default value
        TodaysData2[col] = 536

# Encode non-numeric columns in TodaysData1 using the same encoders
non_numeric_columns_today = TodaysData2.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        unique_values = set(label_encoder.classes_)
        encoded_values = []
        for item in TodaysData2[col]:
            if item in unique_values:
                encoded_values.append(label_encoder.transform([item])[0])
            else:
                encoded_values.append(536)  # Using 536 as a placeholder for unknown categories
        TodaysData2[col] = encoded_values
    else:
        print(f"Warning: Column {col} is not present in the training data.")
        # Fit a new label encoder for columns not present in Train4, but be cautious with this
        label_encoder = LabelEncoder()
        TodaysData2[col] = label_encoder.fit_transform(TodaysData2[col])
        label_encoders[col] = label_encoder

In [68]:
TrainFeatures = Train5.drop(columns = ["K"]).values.reshape(-1, 28)
TrainLabel = Train5["K"].values.reshape(-1, 1)
TodayFeatures = TodaysData2.drop(columns = ["K"]).values.reshape(-1, 28)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 4)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["RFPred"] = RFpred

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [69]:
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        # Handling default value of 536
        TodaysData2[col] = TodaysData2[col].apply(lambda x: label_encoder.inverse_transform([x])[0] if x != 536 else np.nan)

TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_71964/1190844444.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)


In [70]:
TodaysData2.sort_values("RFPred", ascending = True)

,BatterTeam,RoadTeam,HomeTeam,SP,p_throws,Pitches,AB,PA,Outs,HBP,...,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%,RFPred
6,NYY,NYY,BAL,Cade Povich,L,84.5,20.833333,17.333333,12.833333,0.333333,...,4.06,14.4,10.4,4.0,36.69,45.17,17.16,7.89,25.05,3.000000
23,LAA,SEA,LAA,Bryan Woo,R,69.875,18.625,17.75,14.75,0.125,...,3.75,18.12,2.01,16.11,26.83,52.24,16.99,11.27,28.26,3.085526
26,TOR,TOR,AZ,Ryne Nelson,R,81.285714,21.714286,19.857143,14.357143,0.214286,...,3.74,15.13,4.61,10.52,32.51,46.31,15.2,9.49,24.69,3.585526
24,ATL,ATL,SD,Randy Vasquez,R,83.363636,20.727273,18.545455,12.909091,0.181818,...,4.02,15.35,6.14,9.21,35.99,44.82,14.18,9.81,23.99,3.914474
1,DET,LAD,DET,James Paxton,L,84.3125,21.375,18.1875,14.125,0.0625,...,3.94,16.08,11.4,4.68,38.84,42.7,13.71,9.86,23.57,3.940789
19,CWS,PIT,CWS,Marco Gonzales,L,81.333333,23.0,21.0,16.666667,0.0,...,3.54,15.94,7.25,8.69,31.15,47.13,18.44,9.02,27.46,3.953947
3,PHI,OAK,PHI,Hogan Harris,L,90.714286,22.0,19.285714,14.571429,0.142857,...,4.12,17.53,9.09,8.44,37.64,44.72,17.48,10.08,27.56,3.986842
27,AZ,TOR,AZ,Yariel Rodriguez,R,73.571429,17.571429,15.285714,12.428571,0.142857,...,4.19,23.58,11.38,12.2,37.28,47.18,15.92,11.65,27.57,4.000000
5,TB,CLE,TB,Carlos Carrasco,R,78.333333,21.6,19.133333,14.2,0.333333,...,3.63,19.14,6.17,12.97,34.55,45.19,15.74,10.98,26.72,4.000000
21,STL,CHC,STL,Kyle Hendricks,R,74.727273,20.181818,17.636364,11.818182,0.181818,...,3.7,18.02,7.21,10.81,33.94,46.23,18.73,9.37,28.1,4.000000
